# Using Partitioning to Optimize a Circuit

Synthesis is a very powerful circuit optimization technique. However, the input size to even QFAST doesn't scale to larger circuits well. In fact, to be able to synthesize a circuit currently, we will need to be able to simulate it. This will ultimately cap the scaling of synthesis algorithms. However, we can still use a synthesis tool together with partitioner to optimize small blocks of a circuit at a time. BQSKit was designed for this exact use case and in this guide, we will explore how to accomplish this. It is recommended that you read the other tutorials first.

In [1]:
# Load a 16-qubit time evolution circuit generated from the ArQTIC circuit generator.
from bqskit.ir import Circuit

circuit = Circuit.from_file('heisenberg-16-20.qasm')

for gate in circuit.gate_set:
    print(f"{gate} Count:", circuit.count(gate))

HGate Count: 240
RXGate Count: 240
XGate Count: 8
RZGate Count: 180
CNOTGate Count: 360


We will partition the circuit and then use the `ForEachBlockPass` to perform operations on the individual blocks. Note the `ForEachBlockPass` will run the sub tasks in parallel.

In [2]:
from bqskit.compiler import CompilationTask
from bqskit.compiler import Compiler
from bqskit.passes import QuickPartitioner
from bqskit.passes import ForEachBlockPass
from bqskit.passes import QSearchSynthesisPass
from bqskit.passes import ScanningGateRemovalPass
from bqskit.passes import UnfoldPass

# Finally, we construct a compiler and pass the circuit
# and workflow to it.
with Compiler() as compiler:
    synthesized_circuit = compiler.compile(
        circuit,
        [
            QuickPartitioner(3),
            ForEachBlockPass([QSearchSynthesisPass(), ScanningGateRemovalPass()]),
            UnfoldPass(),
        ]
    )

for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

RYGate Count: 292
RXGate Count: 115
U3Gate Count: 134
RZGate Count: 162
CNOTGate Count: 244


**Replace Filters**

The `ForEachBlockPass` takes an optional parameter `replace_filter` that determines if the circuit resulting from running the input passes on the original block should replace the original block. In the below example, we alter the above flow to only replace a block if it has fewer two-qubit gates as a result of running `QSearchSynthesisPass` and `ScanningGateRemovalPass`.

**Exercise:** Try changing the replace filter to suite your needs. You might want to select circuits with greater parallelism: `circuit.parallelism` or choose based on depth `circuit.depth`. 

In [3]:
from bqskit.ir.gates import CXGate

def less_2q_gates(result_circuit, initial_block_as_op):
    begin_cx_count = initial_block_as_op.gate._circuit.count(CXGate())
    end_cx_count = result_circuit.count(CXGate())
    return end_cx_count < begin_cx_count

# Finally, we construct a compiler and pass the circuit
# and workflow to it.
with Compiler() as compiler:
    synthesized_circuit = compiler.compile(
        circuit,
        [
            QuickPartitioner(3),
            ForEachBlockPass(
                [QSearchSynthesisPass(), ScanningGateRemovalPass()],
                replace_filter=less_2q_gates
            ),
            UnfoldPass(),
        ]
    )

for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

RYGate Count: 76
HGate Count: 165
RXGate Count: 179
U3Gate Count: 79
XGate Count: 7
RZGate Count: 139
CNOTGate Count: 236


## Gatesets

Just like we changed the gates used by QSearch in the Search Synthesis tutorial, we can change the gates for the entire circuit using the same method.

**Exercise:** Change the gates used in the below example to change the gate set for the circuit.

In [4]:
from bqskit.ir.gates import ISwapGate, U3Gate
from bqskit.passes.search import SimpleLayerGenerator

layer_gen = SimpleLayerGenerator(two_qudit_gate=ISwapGate(), single_qudit_gate_1=U3Gate())

configured_qsearch_pass = QSearchSynthesisPass(layer_generator=layer_gen)

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(
        circuit,
        [
            QuickPartitioner(3),
            ForEachBlockPass([configured_qsearch_pass, ScanningGateRemovalPass()]),
            UnfoldPass(),
        ]
    )

for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

U3Gate Count: 478
ISwapGate Count: 242


## Block Size

Increasing the partitioner's block size will likely lead to better results at a runtime cost. If you have the computing resources, you can launch a Dask cluster and connect to it via `Compiler()`. The ForEachBlockPass will efficiently distribute the work. See the [Dask documentation](https://docs.dask.org/en/stable/setup.html) for how to launch a cluster.

In [5]:
from bqskit.passes import LEAPSynthesisPass

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(
        circuit,
        [
            QuickPartitioner(4),
            ForEachBlockPass([LEAPSynthesisPass()]),
            UnfoldPass(),
        ]
    )

for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

RYGate Count: 606
RXGate Count: 303
U3Gate Count: 226
RZGate Count: 303
CNOTGate Count: 303


## Iterative Optimization

We have provided support for passes that manage control flow. This enables us to conditionally apply passes or to apply them in a loop. In the below example we will run the partitioning and synthesis sequence in a loop until the circuit stops decreasing in 2-qubit gate count.

In [10]:
from bqskit.compiler import BasePass
from bqskit.passes import WhileLoopPass, GateCountPredicate
from bqskit.ir.gates import CXGate

# Defining a new pass is as easy as implementing a `run` function.
# In this pass, we just print some information about the circuit
class PrintCNOTsPass(BasePass):
    def run(self, circuit, data) -> None:
        print("Current CNOT count:", circuit.count(CXGate()))

with Compiler() as compiler:
    synthesized_circuit = compiler.compile(
        circuit,
        [
            PrintCNOTsPass(),
            WhileLoopPass(
                GateCountPredicate(CXGate()),
                [
                    QuickPartitioner(3),
                    ForEachBlockPass(
                        [
                            QSearchSynthesisPass(),
                            ScanningGateRemovalPass()
                        ],
                        replace_filter=less_2q_gates
                    ),
                    UnfoldPass(),
                    PrintCNOTsPass(),
                ]
            )
        ]
    )

for gate in synthesized_circuit.gate_set:
    print(f"{gate} Count:", synthesized_circuit.count(gate))

RuntimeError: Server connection unexpectedly closed.

There's a lot new in the above example. First, we defined a new pass by subclassing `BasePass` and implementing a `run` method. This pass just prints the number of Controlled-not gates in the circuit when executed. We then use this before and inside a `WhileLoopPass` to see the progress of execution. Second, we perform a `WhileLoopPass` which takes a predicate and a sequence of passes. It will apply the passes supplied until the predicate produces false. We supplied a `GateCountPredicate` which evaluates to False when the specific gate count stops changing.